## Tracking Thresholding using Morris et al (2008)

In [48]:
import numpy as np
from scipy.stats import poisson
from scipy.ndimage.measurements import label
from scipy.stats import mode
import nibabel as nib
from nipype.interfaces import mrtrix as mrt

In [2]:
theDir = "/Users/srothmei/Desktop/charite/toronto/Adalberto/debug/"
csd_file = theDir + "csd8.nii.gz"
csd_file_random = theDir + "csd_random.nii.gz"

singleVoxelTracks = theDir + 'QL_10011_tracks_singleVox.tck'
singleVoxelTracksRandom = theDir + 'QL_10011_tracks_singleVox_random.tck'

singleVoxelTDI = theDir + 'QL_10011_tracks_singleVox_TDI.nii.gz'
singleVoxelRandomTDI = theDir + 'QL_10011_tracks_singleVox_random_TDI.nii.gz'

tracksPerVoxel = 1000
trackPerVoxelRandom = tracksPerVoxel * 20
confidenceThreshold = 95 # Value in %

## Generate non-informative fODF data i.e. fODF data which has equi-distributed directionality information along all possible directions, leaving a sphere

In [3]:
csd8 = nib.load(csd_file)
csd8_data = csd8.get_data()

In [4]:
for i in range(96):
    for j in range(96):
        for k in range(61):
            if np.sum(csd8_data[i,j,k,:] == np.zeros_like(csd8_data[30,40,32,:])) < 45:
                csd8_data[i,j,k,:] = np.zeros_like(csd8_data[30,40,32,:])
                csd8_data[i,j,k,0] = 1.

In [5]:
csd8 = nib.Nifti1Image(csd8_data, csd8.affine, csd8.header)
nib.save(csd8, csd_file_random)

## Now perform fiber tracking for a single-voxel ROI using the (measurement-based) informative CSD map and the random-CSD-map.
While using the random map, the number of tracks-per-voxel is 20x higher to achive statistical power

In [12]:
tracker = mrt.StreamlineTrack()
tracker.inputs.inputmodel = 'SD_PROB'
tracker.inputs.stop = True
tracker.inputs.minimum_tract_length = 30
tracker.inputs.no_mask_interpolation = True
tracker.inputs.step_size = 0.2
tracker.inputs.unidirectional = True #Maybe off?
tracker.inputs.seed_file = theDir + 'seedmask10011_1mm_test.nii.gz'
#tracker.inputs.seed_spec = [-50,-6,23,1]
tracker.inputs.include_file = theDir + 'targetmask1001_1mm.nii.gz'
tracker.inputs.mask_file = theDir + 'wmmask_68_1mm.nii.gz'

# Now first for the "informed case"
tracker.inputs.in_file = csd_file
tracker.inputs.out_file = singleVoxelTracks
tracker.inputs.desired_number_of_tracks = tracksPerVoxel
# Perform the fiber tracking
tracker.run()

# Secondly the "uninformed case"
tracker.inputs.in_file = csd_file_random
tracker.inputs.out_file = singleVoxelTracksRandom
tracker.inputs.desired_number_of_tracks = trackPerVoxelRandom
tracker.run()

   18072 generated,     1000 selected    [100%]
  579438 generated,    20000 selected    [100%]


## Generate maps of the connection probability

In [13]:
tdi = mrt.Tracks2Prob()
tdi.inputs.fraction = False
tdi.inputs.template_file = theDir + 'seedmask10011_1mm.nii.gz'

tdi.inputs.in_file = singleVoxelTracks
tdi.inputs.out_filename = singleVoxelTDI
tdi.run()

tdi.inputs.in_file = singleVoxelTracksRandom
tdi.inputs.out_filename = singleVoxelRandomTDI
tdi.run()

tracks2prob: mapping tracks to image...  100%
tracks2prob: writing image...  100%
tracks2prob: mapping tracks to image...  100%
tracks2prob: writing image...  100%


## Now calculate the Z-Map

In [14]:
tdi_informed = nib.load(singleVoxelTDI)
tdi_informed_data = tdi_informed.get_data()

tdi_random = nib.load(singleVoxelRandomTDI)
tdi_random_data = tdi_random.get_data()

# Equation (4)
meanV = (tracksPerVoxel * tdi_random_data) / float(trackPerVoxelRandom)

# Equation (5)
stdV = np.sqrt(meanV)

## Create Poisson Table
According to the Book of Altman (1999) "Practical Statistics for Medical Research (Page 70)", there is an iterative approach to solve the Possion 
probability mass function. I bet there is also a closed formula for this somewhere but since i dont want to figure this out myself and
werent able to find this formular in the literature i simply do it the "brute-force" way by using a iterative loop

In [24]:
#Quick and dirty loop
Z_Map = np.zeros_like(meanV, dtype='float64')
raw_P_Map = np.zeros_like(Z_Map)
for x in range(np.shape(meanV)[0]):
    for y in range(np.shape(meanV)[1]):
        for z in range(np.shape(meanV)[2]):
            if stdV[x,y,z] > 0.0:
                # Equation (7)
                tmp = (tdi_informed_data[x,y,z] - meanV[x,y,z]) / stdV[x,y,z]
                #if tmp > 0.0:
                #    Z_Map[x,y,z] = np.log(tmp)
                #else:
                Z_Map[x,y,z] = tmp
                #Omit the negative Z-values
                if tmp >= 0.0:
                    k = tdi_informed_data[x,y,z]
                    mu = meanV[x,y,z]
                    raw_P_Map[x,y,z] = 1 - poisson.pmf(k, mu)

# Thresholded P-map
P_Map_thresholded = np.zeros_like(raw_P_Map)
P_Map_thresholded[raw_P_Map >= confidenceThreshold/100.0] = 1

                    
# Some debug/visual stuff
zmapImage = nib.Nifti1Image(Z_Map, tdi_informed.affine, tdi_informed.header)
nib.save(zmapImage, theDir + 'Zmap.nii.gz')

pmapImage = nib.Nifti1Image(raw_P_Map, tdi_informed.affine, tdi_informed.header)
nib.save(pmapImage, theDir + 'Pmap_raw.nii.gz')

pmapThres = nib.Nifti1Image(P_Map_thresholded, tdi_informed.affine, tdi_informed.header)
nib.save(pmapThres, theDir + 'Pmap_thresholded.nii.gz')

## False positive correction
Check if the thresholded voxels have a connection to the seed voxel. If not remove them

In [58]:
structure = np.ones((3,3,3))
tmp = np.zeros_like(P_Map_thresholded)
tmp, bar = label(P_Map_thresholded, structure)
modal_val, modal_count = mode(tmp[tmp>0], axis=None)
P_Map_thresholded_fpCorr = np.zeros_like(P_Map_thresholded)
P_Map_thresholded_fpCorr[tmp == modal_val] = 1

pmapThresFP = nib.Nifti1Image(P_Map_thresholded_fpCorr, tdi_informed.affine, tdi_informed.header)
nib.save(pmapThresFP, theDir + 'Pmap_thresholded_FPcorr.nii.gz')